### Whisper 모델을 내려받아 로컬에서 사용하기

In [ ]:
!pip install transformers datasets[audio] accelerate

In [ ]:
!pip install ipywidgets

In [ ]:
# PyTorch 설치
# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122
!pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu128

In [ ]:
# TorchCodec(미디어 디코딩/인코딩)도 cu128로
!pip install -U "torchcodec==0.6.*" --index-url https://download.pytorch.org/whl/cu128

In [ ]:
# 허깅페이스에서 모델 다운로드
# !git lfs install
# !git clone https://huggingface.co/openai/whisper-large-v3-turbo

In [ ]:
# FFMPEG 설치
# conda-forge에서 FFmpeg 설치 (같은 conda env 안에!)
conda install -n agent-llm -c conda-forge "ffmpeg>=4,<8" -y

In [1]:
# https://huggingface.co/openai/whisper-large-v3-turbo 에 있는 샘플 코드

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True, # 타임스탬프 반환
    chunk_length_s=10,  # 10초 단위로 처리
    stride_length_s=2,  # 2초씩 겹치게 처리
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
sample = "./audio/lsy_audio_2023_58s.mp3"

result = pipe(sample)
# print(result["text"])

print(result)


Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/2

{'text': ' 안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다. GPT-API에 대해서 생소하신 분들도 있을텐데 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요. 그래서 이런 강의들이 사실 많이 있습니다. 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면 GPT로 명확한 미션을 달성하는 챕터 프로그램을 만드는게 사실 쉽지는 않은데 이걸 어떻게 해서 구현을 하는지 그리고 그게 왜 필요한지에 대해서 좀 이야기를 할 거고요. 그 예제로 예제는 여러가지가 될 수 있는데 예제로 하는 것은 음악 플레이리스트 동영상을 자동으로 대화를 통해서 생성하는 프로그램을 만드는 것을 다루려고 합니다. 프로그램이 실행되는 모습을 한번 보여드릴게요. 우리가 만들 프로그램은 이런 식으로 이제 나타나게 되고', 'chunks': [{'timestamp': (0.0, 6.3), 'text': ' 안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다.'}, {'timestamp': (7.18, 10.0), 'text': ' GPT-API에 대해서 생소하신 분들도 있을텐데'}, {'timestamp': (11.0, 17.0), 'text': ' 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서'}, {'timestamp': (17.0, 20.0), 'text': ' 우리가 원하는 프로그램을 어떻게 만드는지에 대해서'}, {'timestamp': (20.0, 22.0), 'text': ' 이야기할 거예요.'}, {'timestamp': (22.0, 24.0), 'text': ' 그래서 이런 강의들이 사실 많이 있습니다.'}, {'timestamp': (24.0, 27.48), 'text': ' 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면'}, {'timestamp': (27.48, 29.58), 'text': ' G

In [2]:
# chunks를 CSV 파일로 저장
start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("lsy_audio_2023_58.csv", index=False, sep="|")
display(df)

,start,end,text
0,0.00,6.30,안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다.
1,7.18,10.00,GPT-API에 대해서 생소하신 분들도 있을텐데
2,11.00,17.00,"우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서"
3,17.00,20.00,우리가 원하는 프로그램을 어떻게 만드는지에 대해서
4,20.00,22.00,이야기할 거예요.
5,22.00,24.00,그래서 이런 강의들이 사실 많이 있습니다.
6,24.00,27.48,그래서 여러가지들이 있는데 이 강의 특징이라고 한다면
7,27.48,29.58,GPT로 명확한 미션을 달성하는
8,29.58,31.66,챕터 프로그램을 만드는게 사실
9,31.66,34.32,쉽지는 않은데 이걸 어떻게 해서
